# 소환사 닉네임과 태그로 계정 정보 가져오기.

In [64]:
# def league_v4_encryptedSummonerId_info(id):
#     queue = "RANKED_SOLO_5x5"
#     url = f"https://kr.api.riotgames.com/lol/league/v4/entries/by-summoner/{id}"
#     response = requests.get(url, headers=header)
    
#     if response.status_code == 200: return response.json()
#     else: print(f"Error fetching data: {response.status_code}"); return None
    

# def account_v1_info(userNickname, tagLine):
#     queue = "RANKED_SOLO_5x5"
#     url = f"https://asia.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{userNickname}/{tagLine}"
#     response = requests.get(url, headers=header)
    
#     if response.status_code == 200: return response.json()
#     else: print(f"Error fetching data: {response.status_code}"); return None

# def summoner_v4_by_encryptedPUUID_info(puuid):
#     queue = "RANKED_SOLO_5x5"
#     url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}"
#     response = requests.get(url, headers=header)
    
#     if response.status_code == 200: return response.json()
#     else: print(f"Error fetching data: {response.status_code}"); return None
    

# player_id = account_v1_info('Justlikethatkr','kr3')
# display(player_id)

# player_id = summoner_v4_by_encryptedPUUID_info(player_id['puuid'])
# display(player_id)

# player = league_v4_encryptedSummonerId_info(player_id['id'])
# display(player)


# API 데이터 불러오기.

## 챌린저 (300명) 티어 정보 가져오기.

In [ ]:
from config0215 import api_key
import requests
 
api_key = api_key
header = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36",
                "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
                "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
                "Origin": "https://developer.riotgames.com",
                "X-Riot-Token": api_key
                }
 
def league_v4_tier_info(tier): # 챌린저 300명 summonerId 불러오기.
    queue = "RANKED_SOLO_5x5"
    url = f"https://kr.api.riotgames.com/lol/league/v4/{tier}leagues/by-queue/{queue}"
    response = requests.get(url, headers=header)
    
    if response.status_code == 200: return response.json()
    else: print(f"Error fetching data: {response.status_code}"); return None
    
    

In [ ]:
tier = 'challenger'
player_json = []

player_json = league_v4_tier_info(tier)['entries']
len(player_json)

300

## 챌린저 puuid 가져오기. (Match 호출을 위함.)

In [96]:
def summoner_v4_summoners_encryptedSummonerId(encryptedSummonerId):
    url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/{encryptedSummonerId}?api_key={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200 : return response.json()
    else : print(f"Error fetching data: {response.status_code}"); return None

In [ ]:
challenger_play = {}
sorted_entries = sorted(player_json, key=lambda x: x['leaguePoints'], reverse=True)

# puuid 가져오기. (match 데이터를 puuid로 가져올 수 있음.) 최대 2분에 100번 api 호출 가능하기 때문에 100명만 기록.
for index, entry in enumerate(sorted_entries[:100], start=1):
    challenger_play[index] = {
        "puuid" : summoner_v4_summoners_encryptedSummonerId(entry['summonerId'])['puuid'],
        "summonerId": entry["summonerId"],
        "LP": entry["leaguePoints"],
        "wins": entry["wins"],
        "losses": entry["losses"]
    }

print((challenger_play)[1])
print(len(challenger_play))

{'puuid': '0RrhGBQ4ePEl6i7hKVQFRV9NL27lbdOKQCVOHcXWRPeaYqPVE9bM6aLeXUITCHblUmoRBUEhvbJaxw', 'summonerId': 'dF9F3NRdYNBKrIwOMMAX3lYTeMOmvM5hRInM5UAALO4DL2U', 'LP': 1997, 'wins': 177, 'losses': 116}
100


## MatchId 정보 가져오기.
한 명의 유저당 최근 20개의 경기.

In [ ]:
def match_v5_matches_bypuuid_puuid_ids(puuid):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=20&api_key={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200 : return response.json()
    else : print(f"Error fetching data: {response.status_code}"); return None

In [124]:
matches = []
for player in challenger_play.values():
    matches.extend(match_v5_matches_bypuuid_puuid_ids(player['puuid']))
    
print(len(matches))

2000


## 2000개 경기 정보 가져오기.

In [144]:
def match_v5_matches_matchid(matchId):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200 : return response.json()
    else : print(f'Error fetching data : {response.status_code}'); return 'error'

In [145]:
import pandas as pd
import time

result_list = []

for i, match in enumerate(matches):
    if i % 100 == 0 and i != 0 : print(len(result_list)); time.sleep(121)
    m = match_v5_matches_matchid(match)['info']['participants']
    if m != 'error':
        data = pd.DataFrame(m)
        result_list.append(data)

len(result_list)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900


2000

In [147]:
df2 = pd.concat(result_list, ignore_index = True)
df2

,PlayerScore0,PlayerScore1,PlayerScore10,PlayerScore11,PlayerScore2,PlayerScore3,PlayerScore4,PlayerScore5,PlayerScore6,PlayerScore7,...,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3,5,5,0,0,42,10,9,25,True
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,5,0,0,45,6,3,8,True
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3,4,5,0,0,30,4,5,9,True
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,3,5,0,0,58,4,9,25,True
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,4,5,0,0,130,22,13,73,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,5,0,0,46,3,4,19,True
19996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,2,5,0,0,47,4,13,7,True
19997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4,6,5,0,0,18,3,3,7,True
19998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4,6,5,0,0,45,6,11,16,True


In [148]:
df2.to_csv('matches_2000.csv', index=False)

In [150]:
df2.columns

Index(['PlayerScore0', 'PlayerScore1', 'PlayerScore10', 'PlayerScore11',
       'PlayerScore2', 'PlayerScore3', 'PlayerScore4', 'PlayerScore5',
       'PlayerScore6', 'PlayerScore7',
       ...
       'turretKills', 'turretTakedowns', 'turretsLost', 'unrealKills',
       'visionClearedPings', 'visionScore', 'visionWardsBoughtInGame',
       'wardsKilled', 'wardsPlaced', 'win'],
      dtype='object', length=145)